Import usefull libreries

In [1]:
from igraph import *
import numpy as np
from math import floor
from random import randint
from operator import itemgetter, attrgetter
from datetime import date
from time import localtime, strftime

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Define the functions to perform the attacks:

In [3]:
# a function to generate the "robust version" of a given graph g
# according to the Barabasi algorithm
# see http://barabasi.com/networksciencebook/chapter/8#buildin-robustness
#     http://barabasi.com/networksciencebook/chapter/8#advanced-8g
def robust_graph(g):
    # compute coefficient A
    g_avg_d = mean(g.degree())
    A = ((2 * g_avg_d ** 2 * (g_avg_d-1)**2) / (2*g_avg_d-1)) ** (float(1)/3)
    g_N = g.vcount()
    
    # compute kmax and kmin according to A
    kmax = int(round(A * (g_N ** (float(2)/3))))
    kmin_low = int(floor((g_N * g_avg_d - kmax) / (g_N - 1)))
    
    # as kmin will be rounded, balance nodes with kmin_low and kmin_low+1 to keep
    # the total number of links and avg degree of the original graph
    rem = ((g_N * g_avg_d - kmax) / (g_N - 1)) - kmin_low
    # 1-rem% of nodes must have degree kmin_low, the rest kmin_low+1
    
    # create the degree sequence to generate the graph
    n_low = int(round((g_N-1) * (1-rem)))
    n_high = (g_N-1) - n_low
    seq_deg_low = [kmin_low for i in range(0,n_low)]
    seq_deg_high = [kmin_low+1 for i in range(0,n_high)]
    seq_deg = seq_deg_high + seq_deg_low
    seq_deg.insert(0,kmax)
    
    # create the "robust" graph
    try:
        g_rob = Graph.Degree_Sequence(seq_deg, method="vl")
        print ("Robust graph generated with method \"vl\"")
    except:
        try:
            g_rob = Graph.Degree_Sequence(seq_deg, method="no_multiple")
            print ("Robust graph generated with method \"no_multiple\"")
        except:
            g_rob = Graph.Degree_Sequence(seq_deg, method="simple")
            print ("Robust graph generated with method \"simple\"")

    return g_rob

In [6]:
DEBUG = True
GRANUL = 1000

sim_attacks = ["targ_deg_sim","targ_btw_sim"]
seq_attacks = ["random", "targ_deg_seq"]#,"targ_btw_seq"]
supported_attacks = sim_attacks + seq_attacks + ['module_based']

# the list of nodes to attack in case of simultaneous attacks of type att_type
# assumes that vertices have a name, and returs the list of vertices to be attacked, by name
def attack_list(g, att_type):
    vert_cent = []
    
    if att_type == "targ_deg_sim":    
        # tuples with names and degrees
        vert_cent = [(v["name"], v.degree()) for idx, v in enumerate(g.vs())]
    elif att_type == "targ_btw_sim": 
        g_vs = g.vs()
        idx = np.argwhere(np.array(g.degree())>0).flatten()
        if(idx.all()):
            sub_g = g.induced_subgraph(idx)
            sub_g_vs = sub_g.vs()
            vert_cent = list(zip(sub_g_vs["name"],sub_g.betweenness()))
    elif att_type == 'module_based':
        gg = g.simplify(multiple=True, loops=True)
        vd = gg.community_fastgreedy()
        vd_clust = vd.as_clustering()
        print("size of communities: {}".format(vd_clust.sizes()))
        edge_cross = np.array(vd_clust.crossing())
        edge_list = np.argwhere(edge_cross == True).flatten()
        between = gg.betweenness()
        for idn in edge_list:
            edge = gg.es[idn]
            name = gg.vs[edge.source]['name']
            b_idx = between[edge.source]
            vert_cent.append((name, b_idx))
        seen = set()
        vert_cent = [(a, b) for a, b in vert_cent if not (a in seen or seen.add(a))] 
    else:
        vert_cent = list(zip(g_vs["name"],g.betweenness()))
#     elif att_type == "targ_btw_sim": 
#         tuples with names and degrees
#         ebs = g.edge_betweenness()
#         ebs_g = [g.es[idx] for idx, eb in enumerate(ebs)]
#         vert_cent = [(g.vs[e.source]["name"], g.vs[e.source].degree()) for e in ebs_g]
        
        
    # sort by the centrality metric in decreasing order
    vert_cent.sort(key=itemgetter(1), reverse=True)
    # no need to kill the last node
    del vert_cent[-1]
    # the IDs of the vertices to be killed, in order of killing
    kill_names = [elem[0] for elem in vert_cent]
    
    return kill_names


# executes a random attack pattern until no node is left, and returns the list of GC proportions
# with respect to the original graph
def attack(g_orig, att_type):
    
    # check that the attack can be done
    if att_type not in supported_attacks:
        print("Attack " + str(att_type) + " is not supported")
        return None

    # deepcopy to avoid modifications to the original graph
    g = deepcopy(g_orig)
    
    # prepare for the output list
    # each element is a tuple in the following form
    # (fraction of removed vertices, fraction of vertices still in the GC)
    initial_size = g.vcount()
    GC_frac = [(0,1)]
    
    # count the # of killed vertices
    killed = 0
    
    # simultaneous attacks
    if att_type in sim_attacks:
        # must refer to vertices by name, as indexes are recomputed after each deletion
        for idx in range(0,g.vcount()):
            g.vs[idx]["name"] = str(idx)
            
        # get the ordered list of vertices to kill (as a list of vertex names)
        kill_list = attack_list(g, att_type)
    
        for kill_name in kill_list: 
            # find the id of the vertex to kill, and kill it
            kill_id = g.vs.find(kill_name).index
            g.delete_vertices(kill_id)
            killed += 1
            # compute the new GC size relative to the original one
            new_GC_size = max(g.clusters().sizes())
            new_point = (float(killed)/initial_size, float(new_GC_size)/initial_size)
            GC_frac.append(new_point)
            if DEBUG and (killed % GRANUL == 0):
                print ("initial_size - killed", new_point)
                if killed % (10*GRANUL) == 0:
                    print ('\n')
             
    
    # sequential attacks
    elif att_type in seq_attacks:
        to_be_killed = g.vcount()
        while to_be_killed > 1:
            # pick the id of the node to attack
            if att_type == "random":
                kill_id = randint(0,to_be_killed-1)
            elif att_type == "targ_deg_seq":
                kill_id = np.argmax(g.degree())
#             elif att_type == "targ_btw_seq":
#                 kill_id = np.argmax(g.edge_betweenness())  
                
            # kill the vertex
            g.delete_vertices(kill_id)
            killed += 1
            to_be_killed -= 1
            # compute the new GC size relative to the original one
            new_GC_size = max(g.clusters().sizes())
            # the new point in the series (fraction of killed nodes, fraction of nodes in the GC)
            new_point = (float(killed)/initial_size, float(new_GC_size)/initial_size)
            GC_frac.append(new_point)
            
            if DEBUG and (killed % GRANUL == 0):
                print ("to_be_killed", new_point)
                if killed % (10*GRANUL) == 0:
                    print ('\n')
    else:
        # module_based attack
        for idx in range(0,g.vcount()):
            g.vs[idx]["name"] = str(idx)
        list_to_kill =  attack_list(g, att_type)
        for kill_name in list_to_kill:
            try:
                kill_id = g.vs.find(kill_name).index
            except Exception as e:
                print(e)
                continue
            g.delete_vertices(kill_id)
            #killed += 1
            #new_GC_size = max(g.clusters().sizes())
            #new_point = (float(killed)/initial_size, float(new_GC_size)/initial_size)
            #GC_frac.append(new_point)
            #if DEBUG and (killed % GRANUL == 0):
            #    print ("initial_size - killed", kill_id)
            #    if killed % (10*GRANUL) == 0:
            #        print ('\n')
        
        print(sorted(g.clusters().sizes(),reverse=True)[:10])
        initial_size = max(g.clusters().sizes())
        new_gc = g.clusters().giant()
        to_be_killed = new_gc.vcount()
        while to_be_killed > 1:
            kill_id = np.argmax(new_gc.degree())
            new_gc.delete_vertices(kill_id)
            killed += 1
            to_be_killed -= 1
            new_GC_size = max(new_gc.clusters().sizes())
            # the new point in the series (fraction of killed nodes, fraction of nodes in the GC)
            new_point = (float(killed)/initial_size, float(new_GC_size)/initial_size)
            GC_frac.append(new_point)
            #new_gc = g.clusters().giant()
            if DEBUG and (killed % GRANUL == 0):
                print ("initial_size - killed", new_point)
                if killed % (10*GRANUL) == 0:
                    print ('\n')

    return GC_frac

# print the resulting robustness data in an ncol format
# <fraction of removed vertices> <fraction of vertices remaining in the GC>
def print_out(lst, file_name):
    try:
        out_file = open(file_name, "w")
        for elem in lst:
            print (elem[0], elem[1], sep = " ", file=out_file)
        out_file.close()
    except:
        print ("Cannot open file for writing output")

In [9]:
# main interface: perform_attacks() is what one should call to attack a graph.
#
# parameters
# - infile is the name of the file with the graph in ncol format
# - directed is whether the graph is directed or not
# - prefix is the subdir+prefix related to the dataset for the name of the outfile, e.g., brightkite/bkite
#   specifically,
#   - the subdir (brightkite in the example) is the folder where the dataset of the graph is available
#   - prefix (bkite in the example) is a prefix string that will be used
#     as the initial part of the name of the output files
#
# The function performs complete attacks of the following kinds
# 1. random
# 2. targeted to the degree, simultaneous
# 3. targeted to the degree, sequential
#
# The three attacks are replicated for the original and the robust version of the graph
#
# For each attack, a txt file is generated using the prefix parameter, with the points of the robustness curve
# Specifically, the output files
# - must be text files
# - in each line they must contain a point of the robustness curve for that particular attack
#   in the following format <fraction of removed vertices> <fraction of vertices remaining in the GC>
# - they will be created in the folder corresponding to the first part of the prefix (until the / character)

def perform_attacks(infile, directed, prefix):
    # load the graph and create its robust version
    g = read(infile, format="ncol", directed=directed)
    g_rob = robust_graph(g)

    # debug print, just to check how long the attacks take to complete
    print ("Random attack to the original graph")
    print (strftime("Start time: %H:%M:%S", localtime()))
    print ("\nPoints in the robustness curve")
    # robustness of the original graph
    # performs the random attack on the graph
    g_orig_frac_random = attack(g, "random")
    # print the resulting robustness curve in one of the output files
    out_file = prefix + "_orig_frac_random.txt"
    print_out(g_orig_frac_random, out_file)
 
    # attack to the original graph
    print ("\n\nTargeted simultaneous attack to the original graph, using degree as centrality metric")
    print (strftime("Start time: %H:%M:%S", localtime()))
    print ("\nPoints in the robustness curve")
    g_orig_frac_deg_sim = attack(g, "targ_deg_sim")
    out_file = prefix + "_orig_frac_deg_sim.txt"
    print_out(g_orig_frac_deg_sim, out_file)
 
    print ("\n\nTargeted simultaneous attack to the original graph, using betweenness as centrality metric")    
    print (strftime("Start time: %H:%M:%S", localtime()))
    print ("\nPoints in the robustness curve")
    g_orig_frac_deg_seq = attack(g, "targ_btw_sim")
    out_file = prefix + "_orig_frac_btw_sim.txt"
    print_out(g_orig_frac_deg_seq, out_file)
    
    print ("\n\nTargeted sequential attack to the original graph, using degree as centrality metric")    
    print (strftime("Start time: %H:%M:%S", localtime()))
    print ("\nPoints in the robustness curve")
    g_orig_frac_deg_seq = attack(g, "targ_deg_seq")
    out_file = prefix + "_orig_frac_deg_seq.txt"
    print_out(g_orig_frac_deg_seq, out_file)
    
#     print ("\n\nTargeted sequential attack to the original graph, using betweenness as centrality metric")    
#     print (strftime("Start time: %H:%M:%S", localtime()))
#     print ("\nPoints in the robustness curve")
#     g_orig_frac_deg_seq = attack(g, "targ_btw_seq")
#     out_file = prefix + "_orig_frac_btw_seq.txt"
#     print_out(g_orig_frac_deg_seq, out_file)
    
    # attack to the robustness graph
    # robustness of the robust version of the graph
    print ("\n\nRandom attack to the robust graph")
    print (strftime("Start time: %H:%M:%S", localtime()))
    print ("\nPoints in the robustness curve")
    g_rob_frac_random = attack(g_rob, "random")
    out_file = prefix + "_rob_frac_random.txt"
    print_out(g_rob_frac_random, out_file)

    print ("\n\nTargeted simultaneous attack to the robust graph, using degree as centrality metric")
    print (strftime("Start time: %H:%M:%S", localtime()))
    print ("\nPoints in the robustness curve")
    g_rob_frac_deg_sim = attack(g_rob, "targ_deg_sim")
    out_file = prefix + "_rob_frac_deg_sim.txt"
    print_out(g_rob_frac_deg_sim, out_file)
    
    print ("\n\nTargeted simultaneous attack to the robust graph, using betweenness as centrality metric")
    print (strftime("Start time: %H:%M:%S", localtime()))
    print ("\nPoints in the robustness curve")
    g_rob_frac_deg_sim = attack(g_rob, "targ_btw_sim")
    out_file = prefix + "_rob_frac_btw_sim.txt"
    print_out(g_rob_frac_deg_sim, out_file)
    
    print ("\n\nTargeted sequential attack to the robust graph, using degree as centrality metric")
    print (strftime("Start time: %H:%M:%S", localtime()))
    print ("\nPoints in the robustness curve")
    g_rob_frac_deg_seq = attack(g_rob, "targ_deg_seq")
    out_file = prefix + "_rob_frac_deg_seq.txt"
    print_out(g_rob_frac_deg_seq, out_file)


    print ("\n\nTargeted module based attack to the original graph")
    print (strftime("Start time: %H:%M:%S", localtime()))
    print ("\nPoints in the robustness curve")
    g_orig_frac_deg_sim = attack(g, "module_based")
    out_file = prefix + "_orig_frac_module_based.txt"
    print_out(g_orig_frac_deg_sim, out_file)

    

    print ("\n\nTargeted module based attack to the robust graph")
    print (strftime("Start time: %H:%M:%S", localtime()))
    print ("\nPoints in the robustness curve")
    g_orig_frac_deg_sim = attack(g_rob, "module_based")
    out_file = prefix + "_robust_frac_module_based.txt"
    print_out(g_orig_frac_deg_sim, out_file)

    # module based
    
#     print ("\n\nTargeted sequential attack to the robust graph, using betweenness as centrality metric")
#     print (strftime("Start time: %H:%M:%S", localtime()))
#     print ("\nPoints in the robustness curve")
#     g_rob_frac_deg_seq = attack(g_rob, "targ_btw_seq")
#     out_file = prefix + "_rob_frac_btw_seq.txt"
#     print_out(g_rob_frac_deg_seq, out_file)

    print (strftime("\n%H:%M:%S\n", localtime()))

In [11]:
# Function to read files with the reobustness points resulting from an attack
import re

# keeps a file name as input, which must be a set of lines with two numbers in each line, separated by white spaces
# returns a tuple composed by two lists
# - the first element is the list of x axes (the list of the numbers in the first position in each line)
# - the second element is the list of y axes (the list of the numbers in the second position in each line)
def read_xy(file):
    x = []
    y = []
    f = open(file, 'rU')
    for line in f:
        strings = re.findall(r'\d+\.?\d*e?[\+-]?\d?\d?', line)
        x.append(float(strings[0]))
        y.append(float(strings[1]))
    f.close()
    return (x,y)

The real work

In [12]:
# Here is the code to start and execute the attacks on the graph - invoke perform_attacks() on the graph under analysis
# Example of attacks - the case of Brightkite

print ("Attacking Facebook\n\n")
perform_attacks("dataset/socfb-wosn-friends.cut.sb.10K.edges", directed=False, prefix="Facebook/fb")

Attacking Facebook


Robust graph generated with method "simple"


Targeted module based attack to the original graph
Start time: 10:15:22

Points in the robustness curve
size of communities: [5104, 33, 3614, 237, 801, 159, 25, 2, 2, 2, 2, 2, 4, 11, 2]
[3493, 1219, 189, 19, 13, 13, 12, 12, 11, 11]
initial_size - killed (0.286286859433152, 0.659032350415116)
initial_size - killed (0.572573718866304, 0.000572573718866304)
initial_size - killed (0.8588605782994561, 0.000286286859433152)


Targeted module based attack to the robust graph
Start time: 10:16:28

Points in the robustness curve
size of communities: [3479, 3088, 2850, 571, 6, 3, 3]
[33, 17, 14, 14, 12, 9, 7, 5, 4, 4]


In [16]:
# read the files with the points of the robustness curves and plot the robustness curves

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.backends.backend_pdf import PdfPages

# read the data from the respective files, putting everything into a dict
# each element in the dict stores the two vectors with the data for the corresponding curve
bk = {}
bk["or_rand"] = read_xy("Facebook/fb_orig_frac_random.txt")
bk["or_deg_seq"] = read_xy("Facebook/fb_orig_frac_deg_seq.txt")
bk["or_deg_sim"] = read_xy("Facebook/fb_orig_frac_deg_sim.txt")
bk["or_btw_sim"] = read_xy("Facebook/fb_orig_frac_btw_sim.txt")

bk["rob_rand"] = read_xy("Facebook/fb_rob_frac_random.txt")
bk["rob_deg_seq"] = read_xy("Facebook/fb_rob_frac_deg_seq.txt")
bk["rob_deg_sim"] = read_xy("Facebook/fb_rob_frac_deg_sim.txt")
bk["rob_btw_sim"] = read_xy("Facebook/fb_rob_frac_btw_sim.txt")

# ‘b’ 	blue
# ‘g’ 	green
# ‘r’ 	red
# ‘c’ 	cyan
# ‘m’ 	magenta
# ‘y’ 	yellow
# ‘k’ 	black
# ‘w’ 	white
#{'b', 'g', 'r', 'c', 'm', 'y', 'k', 'w'};

# plot the 4 curves in the same plot
_ = plt.plot(bk["or_rand"][0], bk["or_rand"][1], 'b-', label = "orig-random")
_ = plt.plot(bk["or_deg_seq"][0], bk["or_deg_seq"][1], 'g-', label = "orig-deg-seq")
_ = plt.plot(bk["or_deg_sim"][0], bk["or_deg_sim"][1], 'r-', label = "orig-deg-sim")
_ = plt.plot(bk["or_btw_sim"][0], bk["or_btw_sim"][1], 'c-', label = "orig-btw-sim")
_ = plt.plot(bk["rob_rand"][0], bk["rob_rand"][1], 'm-', label = "rob-random")
_ = plt.plot(bk["rob_deg_seq"][0], bk["rob_deg_seq"][1], 'y-', label = "rob-deg-seq")
_ = plt.plot(bk["rob_deg_sim"][0], bk["rob_deg_sim"][1], 'k-', label = "rob-deg-sim")
_ = plt.plot(bk["rob_btw_sim"][0], bk["rob_btw_sim"][1], 'w-', label = "rob-btw-sim")
_ = plt.xlabel("% of killed nodes")
_ = plt.ylabel("% of surviving GC")
_ = plt.title("Facebook robustness")
_ = plt.legend()

# save the plot to a PDF file
pdf_file = PdfPages('fb_robustness.pdf')
plt.savefig(pdf_file, format="pdf")
pdf_file.close()
plt.close()

FileNotFoundError: [Errno 2] No such file or directory: 'Facebook/fb_rob_frac_btw_sim.txt'

In [15]:
# read the files with the points of the robustness curves and plot the robustness curves

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.backends.backend_pdf import PdfPages


bk = {}
bk["or_bet_mod"] = read_xy("Facebook/fb_orig_frac_module_based.txt")
bk["rob_bet_mod"] = read_xy("Facebook/fb_robust_frac_module_based.txt")
_ = plt.plot(bk["or_bet_mod"][0], bk["or_bet_mod"][1], 'k--', label = "orig-mod")
_ = plt.plot(bk["rob_bet_mod"][0], bk["rob_bet_mod"][1], 'm--', label = "rob-mod")


_ = plt.xlabel("% of killed nodes")
_ = plt.ylabel("% of surviving GC")
_ = plt.title("Facebook robustness")
_ = plt.legend()
pdf_file = PdfPages('fb_robustness2.pdf')
plt.savefig(pdf_file, format="pdf")
pdf_file.close()
plt.close()